In [ ]:
%pip install -qU langchain langchain-ollama pypdf

In [14]:
import os
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [ ]:
# Load a sample PDF file
pdf_loader = PyPDFLoader("../00-example_data/layout-parser-paper.pdf")  # Replace with actual file path
pdf_docs = pdf_loader.load()
print(f"Loaded {len(pdf_docs)} pages from the file.")

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(pdf_docs)
print(f"Total chunks created: {len(documents)}")


In [ ]:
# Define the Ollama model
model = ChatOllama(model="llama3.2", temperature=0.1)

# Stream the response token by token
chunks = []
for chunk in model.stream("What color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)


In [ ]:
# Stream asynchronously using async for loop
chunks = []
async for chunk in model.astream("What color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)


In [ ]:
# Define a prompt template
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")

# Define an output parser
parser = StrOutputParser()

# Create a streaming chain
chain = prompt | model | parser

# Stream the chain's response
async for chunk in chain.astream({"topic": "parrot"}):
    print(chunk, end="|", flush=True)


In [ ]:
from langchain_core.output_parsers import JsonOutputParser

# Create a chain to stream structured JSON output
json_chain = model | JsonOutputParser()

# Stream the JSON output as it gets generated
async for chunk in json_chain.astream(
    "Provide a JSON list of countries: France, Spain, Japan, with populations."
):
    print(chunk, flush=True)


In [ ]:
# Define a prompt template with correct placeholders
prompt = ChatPromptTemplate.from_template("Answer the question based only on the following context:\n{context}\n\nQuestion: {question}")

# Define the retrieval chain using correct input keys
retrieval_chain = (
    {
        "context": RunnablePassthrough(),  # Simulating retrieval
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

# Stream the response
async for chunk in retrieval_chain.astream({"question": "Where did Harrison work?", "context": "Harrison worked at Kensho"}):
    print(chunk, end="|", flush=True)


In [ ]:
events = []
async for event in model.astream_events("Hello!", version="v2"):
    events.append(event)
    print(event)


In [ ]:
from langchain_core.tools import tool  # Import the tool decorator
from langchain_core.runnables import RunnableLambda

# Define a simple function that reverses a word
def reverse_word(word: str):
    return word[::-1]

# Convert the function into a RunnableLambda
reverse_runnable = RunnableLambda(reverse_word)

# Create a tool that propagates callbacks correctly
@tool
def correct_tool(word: str, callbacks):
    """Reverse the given word and propagate callbacks correctly."""
    return reverse_runnable.invoke(word, {"callbacks": callbacks})

# Stream the tool execution
async for event in correct_tool.astream_events("hello", version="v2"):
    print(event)
